In [2]:
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt

from sentinelhub import (
    SHConfig,
    MimeType,
    CRS,
    BBox,
    SentinelHubRequest,
    SentinelHubDownloadClient,
    DataCollection,
    bbox_to_dimensions,
    DownloadRequest,
    Geometry,
    filter_times,
    MosaickingOrder,
    SentinelHubCatalog
)

import sh_credentials
from utils import plot_image

c:\Users\wjeer\Anaconda3\lib\site-packages\requests\__init__.py:114: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning,


In [3]:
#Connect to Sentinel Hub
config = SHConfig()

#Use sh_credentials in the config
config.instance_id = sh_credentials.instance_id
config.sh_client_id = sh_credentials.sh_client_id
config.sh_client_secret = sh_credentials.sh_client_secret

Found bbox of Zion:
http://bboxfinder.com/#37.161512,-113.167419,37.328589,-112.888298

In [4]:
#Define bbox
#EPSG 4326, WGS 84
zion_coords = [-113.128967,37.164474,-112.920914,37.311603]

#contrails example
#zion_coords = [-113.084129,37.217211,-113.049110,37.250030]

#Define the resolution and extents of the bbox
resolution = 10
zion_bbox = BBox(bbox=zion_coords, crs=CRS.WGS84)
zion_size = bbox_to_dimensions(zion_bbox, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {zion_size} pixels")

Image shape at 10 m resolution: (1881, 1593) pixels
